In [1]:
pip install tensorflow-datasets

Note: you may need to restart the kernel to use updated packages.


In [2]:
#business case-study project 
#code by pranav sankaran
#create a ML algo to determine if a customer will buy again from the auiobook service 
import numpy as np 
import pandas as pd 
from sklearn import preprocessing 
raw_csv_data = np.loadtxt('Audiobooks_data.csv',delimiter=',')
raw_input_data = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]


In [3]:
#balance the dataset 
num_one_targets = int(np.sum(targets_all))

In [4]:
zero_target_counter=0
indeces_to_remove=[]


In [5]:
for i in range(targets_all.shape[0]):
    if targets_all[i]==0:
        zero_target_counter+=1
        if zero_target_counter>num_one_targets:
            indeces_to_remove.append(i)
        
    


In [6]:
unscaled_input_priors=np.delete(raw_input_data,indeces_to_remove,axis=0)
targets_equal_priors = np.delete(targets_all,indeces_to_remove,axis=0)


In [7]:
scaled_inputs=preprocessing.scale(unscaled_input_priors)


In [8]:
#all inputs are now stadardised 
shuffled_indeces = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indeces)

In [9]:
shuffled_inputs= scaled_inputs[shuffled_indeces]
shuffled_targets = targets_equal_priors[shuffled_indeces]

In [10]:
#split the dataset into train , validation , test 
sample_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8*sample_count)
validation_samples_count = int(0.1*sample_count)
test_samples_count = train_samples_count-validation_samples_count

In [11]:
train_inputs = shuffled_inputs[:train_samples_count]

In [12]:
train_targets = shuffled_targets[:train_samples_count]


In [13]:
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

In [14]:
test_inputs=shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count:]
test_targets=shuffled_targets[train_samples_count:train_samples_count+validation_samples_count:]

In [15]:
#save the three datasets in the form of tensors 
np.savez('train_data',inputs=train_inputs,targets=train_targets)
np.savez('validation_data',inputs=validation_inputs,targets=validation_targets)
np.savez('test_data',inputs=test_inputs,targets=test_targets)

In [16]:
import tensorflow as tf


In [17]:
npz=np.load('train_data.npz')

In [18]:
train_inputs=npz['inputs'].astype(np.float)

<ipython-input-18-8775f4c1d395>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_inputs=npz['inputs'].astype(np.float)


In [19]:
train_targets=npz['targets'].astype(np.int)

<ipython-input-19-bd24ff15a260>:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_targets=npz['targets'].astype(np.int)


In [20]:
npz=np.load('validation_data.npz')
validation_train,validation_test=npz['inputs'].astype(np.float),npz['targets'].astype(np.int)

<ipython-input-20-6f063185b271>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  validation_train,validation_test=npz['inputs'].astype(np.float),npz['targets'].astype(np.int)
<ipython-input-20-6f063185b271>:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release

In [21]:
npz=np.load('test_data.npz')
validation_train,validation_test=npz['inputs'].astype(np.float),npz['targets'].astype(np.int)

<ipython-input-21-7edd0a842966>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  validation_train,validation_test=npz['inputs'].astype(np.float),npz['targets'].astype(np.int)
<ipython-input-21-7edd0a842966>:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release

In [22]:
input_size = 10
output_size = 2
hidden_layer_size = 75

In [24]:
model=tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax')
])

In [25]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


In [29]:
batch_size=100
max_epochs=100
callbacks=tf.keras.callbacks.EarlyStopping(patience=3)


model.fit(train_inputs,train_targets,batch_size=batch_size,callbacks=[callbacks],validation_data=(validation_inputs,validation_targets),verbose=2,epochs=max_epochs)

Epoch 1/100
36/36 - 0s - loss: 0.2855 - accuracy: 0.8419 - val_loss: 0.3169 - val_accuracy: 0.8188 - 101ms/epoch - 3ms/step
Epoch 2/100
36/36 - 0s - loss: 0.2884 - accuracy: 0.8402 - val_loss: 0.3208 - val_accuracy: 0.8277 - 73ms/epoch - 2ms/step
Epoch 3/100
36/36 - 0s - loss: 0.2867 - accuracy: 0.8396 - val_loss: 0.3298 - val_accuracy: 0.8054 - 68ms/epoch - 2ms/step
Epoch 4/100
36/36 - 0s - loss: 0.2915 - accuracy: 0.8379 - val_loss: 0.3281 - val_accuracy: 0.8188 - 72ms/epoch - 2ms/step


In [30]:
test_loss,test_accuracy=model.evaluate(test_inputs,test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3281 - accuracy: 0.8188
